# <h1 align="center">COSC2753 - Machine Learning</h1>

<h2 align="center">Final Prediction</h2>

# TABLE OF CONTENT



# IMPORT LIBRARIES

In [1]:
import os
import pandas as pd
import numpy as np
import importlib
import tensorflow as tf
# Import custom libraries
import utils.visualizations as vi

importlib.reload(vi);

In [24]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from keras.models import load_model
from keras.src.legacy.preprocessing.image import ImageDataGenerator

# Setup Necessary Variables

In [3]:
tf.keras.utils.set_random_seed(42)  

OUTPUT_CLASSES = 10
BATCH_SIZE = 32
INPUT_DIM = (128, 128, 3)

In [4]:
# File paths
test_df_path = "./inputs/prediction_submission.csv"
output_file = "./inputs/COSC2753_A2_S2_G2.csv"
test_dir = "./inputs/test_images"

In [5]:
original_test_df = pd.read_csv(test_df_path)

test_df = original_test_df.copy()
train_df = pd.read_csv("./inputs/training_df.csv")

# Add full image paths
test_df["image_path"] = test_df["image_id"].apply(lambda x: os.path.join(test_dir, x))

test_df

,image_id,label,variety,age,image_path
0,200001.jpg,NaN,NaN,NaN,./inputs/test_images\200001.jpg
1,200002.jpg,NaN,NaN,NaN,./inputs/test_images\200002.jpg
2,200003.jpg,NaN,NaN,NaN,./inputs/test_images\200003.jpg
3,200004.jpg,NaN,NaN,NaN,./inputs/test_images\200004.jpg
4,200005.jpg,NaN,NaN,NaN,./inputs/test_images\200005.jpg
...,...,...,...,...,...
3464,203465.jpg,NaN,NaN,NaN,./inputs/test_images\203465.jpg
3465,203466.jpg,NaN,NaN,NaN,./inputs/test_images\203466.jpg
3466,203467.jpg,NaN,NaN,NaN,./inputs/test_images\203467.jpg
3467,203468.jpg,NaN,NaN,NaN,./inputs/test_images\203468.jpg


# Setup Submission Data Frame

In [6]:
submission_df = test_df.copy()
submission_df.drop(columns=["image_path"], inplace=True)
submission_df

,image_id,label,variety,age
0,200001.jpg,NaN,NaN,NaN
1,200002.jpg,NaN,NaN,NaN
2,200003.jpg,NaN,NaN,NaN
3,200004.jpg,NaN,NaN,NaN
4,200005.jpg,NaN,NaN,NaN
...,...,...,...,...
3464,203465.jpg,NaN,NaN,NaN
3465,203466.jpg,NaN,NaN,NaN
3466,203467.jpg,NaN,NaN,NaN
3467,203468.jpg,NaN,NaN,NaN


In [7]:
def load_and_prepare_model(
    model_path, test_df, is_resized=True, target_size=(128, 128)
):
    # Load training data
    if not os.path.exists(model_path):
        print(f"[ERROR] Model not found at {model_path}")
        return None, None, None

    # Load the model
    if not os.path.exists(model_path):
        print("[ERROR] Model not found at {model_path}")
        return None, None, None

    model = load_model(model_path)
    print(f"Model loaded from {model_path}")

    datagen = ImageDataGenerator(
        rescale=1.0 / 255.0 if is_resized else None, data_format="channels_last"
    )

    # Create a generator for predictions
    generator = datagen.flow_from_dataframe(
        dataframe=test_df,
        directory="",
        x_col="image_path",
        y_col=None,
        target_size=target_size,
        batch_size=BATCH_SIZE,
        class_mode=None,
        shuffle=False,
    )

    return model, generator

In [8]:
def wrap_generator(gen):
    for batch in gen:
        yield (batch,)  

# Task 1: Disease Classification Final Prediction

In [10]:
disease_model_path = "./models/vgg_best.keras"
disease_classes = {
    0: "bacterial_leaf_blight",
    1: "bacterial_leaf_streak",
    2: "bacterial_panicle_blight",
    3: "blast",
    4: "brown_spot",
    5: "dead_heart",
    6: "downy_mildew",
    7: "hispa",
    8: "normal",
    9: "tungro",
}


disease_model, disease_generator = load_and_prepare_model(
    disease_model_path,
    test_df,
)

Model loaded from ./models/vgg_best.keras
Found 3469 validated image filenames.


In [11]:
vi.print_header("Disease Final Prediction")
disease_generator.reset()
disease_pred_probs = disease_model.predict(
    wrap_generator(disease_generator), steps=len(disease_generator), verbose=1
)

print("Predicted probabilities shape:", disease_pred_probs.shape)

disease_pred_indices = np.argmax(disease_pred_probs, axis=1)
disease_labels = [disease_classes[i] for i in disease_pred_indices]

+----------------------------------------------+
|           DISEASE FINAL PREDICTION           |
+----------------------------------------------+
109/109 ━━━━━━━━━━━━━━━━━━━━ 38s 343ms/step
Predicted probabilities shape: (3469, 10)


## Save to data frame

In [15]:
submission_df["label"] = disease_labels
submission_df

,image_id,label,variety,age
0,200001.jpg,hispa,NaN,NaN
1,200002.jpg,normal,NaN,NaN
2,200003.jpg,blast,NaN,NaN
3,200004.jpg,blast,NaN,NaN
4,200005.jpg,blast,NaN,NaN
...,...,...,...,...
3464,203465.jpg,dead_heart,NaN,NaN
3465,203466.jpg,hispa,NaN,NaN
3466,203467.jpg,normal,NaN,NaN
3467,203468.jpg,blast,NaN,NaN


# Task 2: Variety Classification Final Prediction

In [32]:
variety_model_path = "./models/efficientnet_final.keras"
variety_classes = {
    0: "ADT45",
    1: "AndraPonni",
    2: "AtchayaPonni",
    3: "IR20",
    4: "KarnatakaPonni",
    5: "Onthanel",
    6: "Ponni",
    7: "RR",
    8: "Surya",
    9: "Zonal",
}

variety_model, variety_generator = load_and_prepare_model(
    variety_model_path, test_df, is_resized=False
)

Model loaded from ./models/efficientnet_final.keras
Found 3469 validated image filenames.


In [33]:
vi.print_header("Variety Final Prediction")

variety_generator.reset()
variety_pred_probs = variety_model.predict(
    wrap_generator(variety_generator), steps=len(variety_generator), verbose=1
)

print("Predicted probabilities shape:", variety_pred_probs.shape)

variety_pred_indices = np.argmax(variety_pred_probs, axis=1)
variety_labels = [variety_classes[i] for i in variety_pred_indices]

+----------------------------------------------+
|           VARIETY FINAL PREDICTION           |
+----------------------------------------------+
109/109 [==============================] - 31s 266ms/step
Predicted probabilities shape: (3469, 10)


## Save to data frame

In [34]:
submission_df["variety"] = variety_labels
submission_df

,image_id,label,variety,age
0,200001.jpg,hispa,ADT45,NaN
1,200002.jpg,normal,ADT45,NaN
2,200003.jpg,blast,Surya,NaN
3,200004.jpg,blast,Surya,NaN
4,200005.jpg,blast,AndraPonni,NaN
...,...,...,...,...
3464,203465.jpg,dead_heart,ADT45,NaN
3465,203466.jpg,hispa,Surya,NaN
3466,203467.jpg,normal,ADT45,NaN
3467,203468.jpg,blast,Surya,NaN


# Task 3: Age Final Prediction

In [10]:
agent_model_path = "./models/resnet50v2_best_model.keras"

age_model, age_generator = load_and_prepare_model(
    agent_model_path, test_df
)

Model loaded from ./models/resnet50v2_best_model.keras
Found 3469 validated image filenames.


In [12]:
vi.print_header("Age Final Prediction")
age_pred = age_model.predict(
    wrap_generator(age_generator), steps=len(age_generator), verbose=1
).flatten()

age_pred = np.round(age_pred)

+------------------------------------------+
|           AGE FINAL PREDICTION           |
+------------------------------------------+
109/109 ━━━━━━━━━━━━━━━━━━━━ 46s 420ms/step


## Save to data frame

In [13]:
submission_df["age"] = age_pred
submission_df

,image_id,label,variety,age
0,200001.jpg,hispa,ADT45,67.0
1,200002.jpg,normal,ADT45,69.0
2,200003.jpg,blast,Surya,70.0
3,200004.jpg,blast,Surya,45.0
4,200005.jpg,blast,AndraPonni,76.0
...,...,...,...,...
3464,203465.jpg,dead_heart,ADT45,66.0
3465,203466.jpg,hispa,Surya,68.0
3466,203467.jpg,normal,ADT45,70.0
3467,203468.jpg,blast,Surya,66.0


# Save to CSV

In [15]:
vi.print_header("Save to CSV")
submission_df.to_csv(output_file, index=False)
print(f"Submission file saved to {output_file}")

submission_df

+---------------------------------+
|           SAVE TO CSV           |
+---------------------------------+
Submission file saved to ./inputs/COSC2753_A2_S2_G2.csv


,image_id,label,variety,age
0,200001.jpg,hispa,ADT45,67.0
1,200002.jpg,normal,ADT45,69.0
2,200003.jpg,blast,Surya,70.0
3,200004.jpg,blast,Surya,45.0
4,200005.jpg,blast,AndraPonni,76.0
...,...,...,...,...
3464,203465.jpg,dead_heart,ADT45,66.0
3465,203466.jpg,hispa,Surya,68.0
3466,203467.jpg,normal,ADT45,70.0
3467,203468.jpg,blast,Surya,66.0
